In [1]:
import json
import pandas as pd
import dill as pickle

class DataUtil2:
    """"""
    @classmethod
    def load(cls, file, format, date_cols= None):
        """"""
        data = ''
        if(format== 'csv'):
            data = pd.read_csv(file, parse_dates= date_cols)
        elif(format== 'json'):
            with open(file, 'r') as i_file:
                data = json.load(file)
            i_file.close()
        elif(format== 'pkl'):
            with open(file, 'rb') as i_file:
                data = pickle.load(i_file)
            i_file.close()
        elif(format == 'hdf'):
            data = pd.read_hdf(path_or_buf= file, key='undefined')

        return  data

    @classmethod
    def save(cls, data, file, format, precision= 8):
        """"""
        if(format == 'csv'):
            data.to_csv(file, float_format= '%%.%df' % precision, index= False)
        elif(format == 'json'):
            with open(file, 'w') as o_file:
                json.dump(data, o_file, ensure_ascii= True, indent= 4)
            o_file.close()
        elif(format == 'pkl'):
            with open(file, 'wb') as o_file:
                pickle.dump(data, o_file, -1)
            o_file.close()
        elif(format== 'hdf'):
            data.to_hdf(path_or_buf= file, key='undefined', mode='w', complib='blosc')

        return

In [2]:
#######################
# LigthGBM Regression #
#######################
import numpy as np
import pandas as pd
import datetime
import time
import os,sys
import gc
from sklearn import *
import lightgbm
import random
import json

drop_cols = ['id', 'visit_date', 'visitors', 'hpg_store_id', 'fold', 'air_store_id']

cate_cols = ['store_id_encoded', 'area_name', 'city', 'genre_name']
cate_feats = ['dow', 'hol_days', 'day', 'pom', 'prev_is_holiday', 'next_is_holiday', 
              'wom', 'woy', 'is_weekends', 'holiday_flg', 'month', 'is_up_corner']
for mod in ['air', 'hpg']:
    cate_feats.extend(['%s_%s' % (mod, c) for c in cate_cols])
    
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

DataBaseDir = '../../data'
InputDir = '%s/l0/kfold' % DataBaseDir
kfold = 5
strategy = 'lgb_l2'
#### load data
valid_dfs = []
holdout_dfs = []
test_dfs = []
trees = ['NN_EF']
for fold in range(kfold):
    FoldInputDir = '%s/%s' % (InputDir, fold)
    valid = pd.read_csv('%s/valid.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    holdout = pd.read_csv('%s/holdout.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    test = pd.read_csv('%s/test.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    for t in trees:
        # load cb_ef
        FoldOutputDir = '%s/l1/kfold/%s' % (DataBaseDir, fold)
        valid_cb_ef = pd.read_csv('%s/valid_%s.csv' % (FoldOutputDir, t), parse_dates= ['visit_date']).reset_index(drop= True)
        holdout_cb_ef = pd.read_csv('%s/holdout_%s.csv' % (FoldOutputDir, t), parse_dates= ['visit_date']).reset_index(drop= True)
        test_cb_ef = pd.read_csv('%s/test_%s.csv' % (FoldOutputDir, t), parse_dates= ['visit_date']).reset_index(drop= True)
        # concate
        valid = pd.concat([valid, valid_cb_ef[[t]]], axis= 1)
        holdout = pd.concat([holdout, holdout_cb_ef[[t]]], axis= 1)
        test = pd.concat([test, test_cb_ef[[t]]], axis= 1)
    #
    valid['fold'] = fold
    valid_dfs.append(valid)
    holdout_dfs.append(holdout)
    test_dfs.append(test)
TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True)
##### model selection with CV
# score
holdout_score = .0
# parameters
params = {
    "boosting": "gbdt",
    "objective": "regression_l2",
    "lambda_l2": [200, 80, 20],
    "learning_rate": [0.4, 0.2, 0.1],
    "max_depth": [12, 16, 20],
    
#     "lambda_l2": [20],
#     "learning_rate": [0.2],
#     "max_depth": [16],
#     "device": "gpu",
    
    "num_iterations": 150,
    "min_data_in_leaf": 20,
    'num_leaves': 255,

    "feature_fraction": 0.90,
    "bagging_fraction": 0.85,
    "bagging_freq": 20,
    "min_hessian": 0.001,

    "max_bin": 63,
}
total_cols = [c for c in TrainData.columns if c not in drop_cols]
K = int(0.6 * len(total_cols))
N = 200


start = time.time()
for idx in range(N): 
    selected_cols = [total_cols[i] for i in sorted(random.sample(range(len(total_cols)), K))]
    OutputDir = '%s/MM2/l1/%s' % (DataBaseDir, idx)
    if(os.path.exists(OutputDir) == False):
        os.makedirs(OutputDir)
    # save feature space
    with open('%s/sub_feats.txt' % OutputDir, 'w') as o_file:
        for feat in selected_cols:
            o_file.write('%s\n' % feat)
    o_file.close()
    ##
    BestParmas = {}
    BestScore = 1.0
    cv_score = .0
    for l2 in params['lambda_l2']:
        for lr in params['learning_rate']:
            for depth in params['max_depth']:
                cv_rmsle = .0
                for fold in range(kfold):
                    FoldData = {
                        'train': TrainData[TrainData['fold'] != fold],
                        'valid': TrainData[TrainData['fold'] == fold]
                    }
                    d_cv = lightgbm.Dataset(FoldData['train'][selected_cols], 
                                        label= FoldData['train']['visitors'], 
                                        #max_bin= params['max_bin'], 
                                        silent= True, 
                                        free_raw_data= True)
                    param = {
                        'boosting': 'gbdt',
                        'objective': 'regression_l2',

                        'lambda_l2': l2,
                        'learning_rate': lr,
                        'max_depth': depth,

                        'num_iterations': params['num_iterations'],
                        'feature_fraction': params['feature_fraction'],
                        'bagging_fraction': params['bagging_fraction'],
                        'bagging_freq': params['bagging_freq'],
                        'min_hessian': params['min_hessian'],
                        'max_bin': params['max_bin'],
                    }
                    model = lightgbm.train(param, d_cv)
                    FoldData['valid'][strategy] = model.predict(FoldData['valid'][selected_cols])
                    rmsle_valid = RMSLE(FoldData['valid']['visitors'], FoldData['valid'][strategy])
                    cv_rmsle += rmsle_valid
                cv_rmsle /= kfold
                if(cv_rmsle < BestScore):
                    BestScore = cv_rmsle
                    BestParmas['lambda_l2'] = l2
                    BestParmas['learning_rate'] = lr
                    BestParmas['max_depth'] = depth
                end = time.time()
                print('running for params, l2 %s, lr %s, depth %s done, cv score %.5f, time elapsed %.2fs' % (l2, lr, depth, cv_rmsle, (end - start)))
    end = time.time()
    print('grid search done, time elapsed %.2fs' % (end - start))
    print('best params, %s' % BestParmas)
    ## retrain and store
    cv_score = .0
    holdout_score = .0
    for fold in range(kfold):
        FoldData = {
            'train': TrainData[TrainData['fold'] != fold],
            'valid': TrainData[TrainData['fold'] == fold],
            'holdout': holdout_dfs[fold],
            'test': test_dfs[fold]
        }
        # train
        d_cv = lightgbm.Dataset(FoldData['train'][selected_cols], 
                            label= FoldData['train']['visitors'].values, 
                            #max_bin= params['max_bin'], 
                            silent= True, 
                            free_raw_data= True)
        param = {
            'boosting': 'gbdt',
            'objective': 'regression_l2',
                
            'lambda_l2': BestParmas['lambda_l2'],
            'learning_rate': BestParmas['learning_rate'],
            'max_depth': BestParmas['max_depth'],
                        
            'num_iterations': params['num_iterations'],
            'feature_fraction': params['feature_fraction'],
            'bagging_fraction': params['bagging_fraction'],
            'bagging_freq': params['bagging_freq'],
            'min_hessian': params['min_hessian'],
            'max_bin': params['max_bin'],
        }
        model = lightgbm.train(param, d_cv)
        # for valid
        FoldData['valid'][strategy] = model.predict(FoldData['valid'][selected_cols])
        rmsle_valid = RMSLE(FoldData['valid']['visitors'].values, FoldData['valid'][strategy])
        cv_score += rmsle_valid
        # for holdout
        FoldData['holdout'][strategy] = model.predict(FoldData['holdout'][selected_cols])
        rmsle_holdout = RMSLE(FoldData['holdout']['visitors'].values, FoldData['holdout'][strategy])
        holdout_score += rmsle_holdout
        # for test
        FoldData['test'][strategy] = model.predict(FoldData['test'][selected_cols])

        print('fold %s: valid score %.6f, holdout score %.6f, valid length %s' % (fold, rmsle_valid, rmsle_holdout, len(FoldData['valid'])))  
        #### output
        FoldOutputDir = '%s/kfold/%s' % (OutputDir, fold)
        if(os.path.exists(FoldOutputDir) == False):
            os.makedirs(FoldOutputDir)
        for mod in FoldData.keys():
            if(mod == 'train'):
                continue
            OutCols = []
            if(mod == 'test'):
                OutCols.append('id')
            OutCols.extend(['air_store_id', 'visit_date', 'visitors', strategy])
            OutputFile = '%s/%s_%s.csv' % (FoldOutputDir, mod, strategy)
            OutFoldData = FoldData[mod][OutCols]
            OutFoldData.to_csv(OutputFile, index= False)
    
    cv_score /= kfold # Average valid set predictions
    holdout_score /= kfold # Average holdout set predictions

    end = time.time()
    print('\n======================')
    print("CV score %.4f, Holdout score %.4f, Elapsed time: %.2fs" % (cv_score, holdout_score, (end - start)))
    print('======================\n')
    
    with open('%s/result.txt' % OutputDir, 'w') as o_file:
        o_file.write('%s\n' % json.dumps(BestParmas))
        o_file.write('cv score %.4f, holdout score %.4f' % (cv_score, holdout_score))
    o_file.close()
    
    end = time.time()
    print('tree %s done, time elapsed %.2f' % (idx, (end - start)))
    
#     if(idx == 0):
#         break

/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module

running for params, l2 200, lr 0.4, depth 12 done, cv score 0.50088, time elapsed 19.38s
running for params, l2 200, lr 0.4, depth 16 done, cv score 0.50112, time elapsed 38.41s


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-f3df39f31540>", line 132, in <module>
    model = lightgbm.train(param, d_cv)
  File "/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/lightgbm/engine.py", line 199, in train
    booster.update(fobj=fobj)
  File "/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/lightgbm/basic.py", line 1507, in update
    ctypes.byref(is_finished)))
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1806, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above excep

KeyboardInterrupt: 